<a href="https://colab.research.google.com/github/TayJen/H_Tatarsan2022/blob/main/H_Tatarstan2022_base.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Импорт библиотек + необходимые установки

In [1]:
%cd /content/drive/MyDrive/H_Tatarstan2022/
!pwd

/content/drive/MyDrive/H_Tatarstan2022
/content/drive/MyDrive/H_Tatarstan2022


In [2]:
# !unzip data/zipped/train_dataset_train.zip -d data/
# !unzip data/zipped/test_dataset_test.zip -d data/

In [3]:
# !git clone https://github.com/ultralytics/yolov5.git
!pip install -r yolov5/requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 596 kB 20.2 MB/s 
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [4]:
import os
import pandas as pd
import numpy as np

np.random.seed(59)

### Пример решения

In [5]:
path_sample = 'data/sample_solution.csv'
df_sample = pd.read_csv(path_sample, sep=';')
df_sample.head(15)

,ImageID,LabelName,Conf,XMin,XMax,YMin,YMax
0,24-11-2021_05-31-51_AM.jpg,3,0.825893,0.035938,0.053646,0.305556,0.373148
1,25-11-2021_06-10-03_AM.jpg,3,0.917864,0.718750,0.763392,0.169737,0.357895
2,24-11-2021_09-05-00_AM.jpg,3,0.536900,0.571484,0.587500,0.320139,0.389583
3,24-11-2021_09-05-00_AM.jpg,3,0.712740,0.873438,0.892578,0.313195,0.398611
4,24-11-2021_09-05-00_AM.jpg,3,0.770197,0.895703,0.915235,0.325695,0.408333
5,24-11-2021_09-05-00_AM.jpg,3,0.786203,0.682031,0.700391,0.306945,0.395833
6,24-11-2021_09-05-00_AM.jpg,3,0.829074,0.642969,0.670313,0.422222,0.554166
7,24-11-2021_02-42-46_PM.jpg,3,0.529614,0.328497,0.347471,0.144737,0.223027
8,24-11-2021_02-42-46_PM.jpg,3,0.827705,0.313244,0.332218,0.136184,0.226974
9,23-11-2021_05-36-22_PM.jpg,3,0.872894,0.205729,0.242709,0.263889,0.365741


## Подготовка данных

In [6]:
import os

len(os.listdir('data/train/images')), len(os.listdir('data/test/images'))

(2002, 855)

In [7]:
# Распределение объектов в train
pd.Series([file_name.split('_')[-1]
           for file_name in os.listdir('data/train/labels')]).value_counts()

head..txt        1741
human..txt       1740
car..txt         1647
face..txt         630
carplate..txt     246
dtype: int64

### Yolo

In [8]:
# Создадим обучающий датасет в формате yolo
# !mkdir data_for_yolo
# !mkdir data_for_yolo/data
# !mkdir data_for_yolo/data/images
# !mkdir data_for_yolo/data/labels
# !mkdir data_for_yolo/data/images/train
# !mkdir data_for_yolo/data/labels/train
# !mkdir data_for_yolo/data/images/test
# !mkdir data_for_yolo/data/labels/test

In [29]:
import yaml

def create_yaml_cont():
    """
        train: data_for_yolo/data/images/train/
        val: data_for_yolo/data/images/test/

        # number of classes
        nc: 5

        # class names
        names: ['human', 'head', 'face', 'car', 'carplate']
    """

    yaml_content = {
        'train': '../data_for_yolo/data/images/train/',
        'val': '../data_for_yolo/data/images/test/',
        'nc': 5,
        'names': ['human', 'head', 'face', 'car', 'carplate']
    }

    with open('data_for_yolo/dataset.yaml', 'w') as f_yml:
        yaml.dump(yaml_content, f_yml)

# Чтобы выполнить надо расскомментить
create_yaml_cont()

In [10]:
# Сделаем разметку как в Yolo, после этого нужным образом перемешаем и
# закинем в папку data_for_yolo

name_to_class_num = {
    'human': 0,
    'head': 1,
    'face': 2,
    'car': 3,
    'carplate': 4
}

for img_name in os.listdir('data/train/images'):
    print(img_name.split('.')[0])
    break

25-11-2021_06-28-29_AM


In [11]:
for img_name in os.listdir('data/train/images/'):
    img_name = img_name.split('.')[0]
    print(img_name)
    for f in os.listdir('data/train/labels/'):
        if f.startswith(img_name):
            print(f)
    break

25-11-2021_06-28-29_AM
25-11-2021_06-28-29_AM_human..txt
25-11-2021_06-28-29_AM_head..txt
25-11-2021_06-28-29_AM_face..txt
25-11-2021_06-28-29_AM_car..txt


In [12]:
for img_name in os.listdir('data/train/images/'):
    img_name = img_name.split('.')[0]
    print(img_name)
    for f in os.listdir('data/train/labels/'):
        if f.startswith(img_name):
            print(f.split('.')[0].split('_')[-1])
            with open('data/train/labels/' + f, 'r') as f_old:
                print(f_old.readlines())
    break

25-11-2021_06-28-29_AM
human
['0 0.5355512152777778 0.7598996913580247 0.08703124999999999 0.34498456790123466\n', '0 0.6109288194444444 0.809579475308642 0.06459201388888886 0.2568132716049384\n']
head
['0 0.6060763888888889 0.7109413580246914 0.027326388888888904 0.05478395061728395\n', '0 0.5411002604166666 0.6192052469135803 0.02878038194444442 0.060462962962962975\n']
face
['0 0.6046875 0.7285956790123457 0.017083333333333388 0.023487654320987695\n', '0 0.5391970486111111 0.6480131172839506 0.018220486111111118 0.03158179012345675\n']
car
['0 0.08944661458333333 0.24220679012345678 0.17889322916666667 0.2080246913580247\n', '0 0.15489149305555555 0.0732716049382716 0.14311631944444445 0.11594135802469135\n']


In [13]:
!mkdir data/train/yolo_labels

mkdir: cannot create directory ‘data/train/yolo_labels’: File exists


In [14]:
def yolo_annotation():
    old_path = 'data/train/labels/'
    new_path = 'data/train/yolo_labels/'

    for img_name in os.listdir('data/train/images/'):
        img_name = img_name.split('.')[0]
        print(img_name)
        with open(new_path + img_name + '.txt', 'w') as f_new:
            for f in os.listdir(old_path):
                if f.startswith(img_name):
                    class_name = f.split('.')[0].split('_')[-1]
                    class_num = name_to_class_num[class_name]
                    with open(old_path + f, 'r') as f_old:
                        for row in f_old.readlines():
                            f_new.write(str(class_num) + row[1:])

# yolo_annotation()

### Shuffle + Yolo

In [15]:
images_names = os.listdir('data/train/images')
print(len(images_names))

2002


In [16]:
from sklearn.model_selection import train_test_split

train_images, test_images = train_test_split(images_names, test_size=0.2,
                                             shuffle=True, random_state=59)

len(train_images), len(test_images)

(1601, 401)

In [17]:
type(train_images)

list

In [18]:
train_images = set(train_images)
test_images = set(test_images)

len(train_images), len(test_images)

(1601, 401)

In [19]:
from shutil import copy


def copy_to_yolo():
    old_path_img = 'data/train/images/'
    old_path_lbl = 'data/train/yolo_labels/'
    yolo_path_img = 'data_for_yolo/data/images/'
    yolo_path_lbl = 'data_for_yolo/data/labels/'

    for image_name in os.listdir(old_path_img):
        lbl_name = image_name.split('.')[0] + '.txt'
        print(image_name, lbl_name)
        if image_name in train_images:
            copy(old_path_img + image_name, yolo_path_img + 'train/' + image_name)
            copy(old_path_lbl + lbl_name, yolo_path_lbl + 'train/' + lbl_name)
        elif image_name in test_images:
            copy(old_path_img + image_name, yolo_path_img + 'test/' + image_name)
            copy(old_path_lbl + lbl_name, yolo_path_lbl + 'test/' + lbl_name)

# Чтобы выполнить надо расскоментить
# copy_to_yolo()

In [20]:
len(os.listdir('data_for_yolo/data/images/train')), len(os.listdir('data_for_yolo/data/images/test'))

(1601, 401)

## Model

In [22]:
!nvidia-smi

Mon Aug  8 15:20:37 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   58C    P8    11W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [24]:
path_to_data = 'data_for_yolo/dataset.yaml'
# --freeze 10 подразумевает заморозку первых 10 блоков сети

In [30]:
!python ./yolov5/train.py --img 416 --batch 16 --epochs 100 --freeze 10 --data {path_to_data} --weights ./yolov5/yolov5s.pt --name baseline_yolov5s_results --cache

train: weights=./yolov5/yolov5s.pt, cfg=, data=data_for_yolo/dataset.yaml, hyp=yolov5/data/hyps/hyp.scratch-low.yaml, epochs=100, batch_size=16, imgsz=416, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=yolov5/runs/train, name=baseline_yolov5s_results, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[10], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: ⚠️ YOLOv5 is out of date by 4 commits. Use `git pull` or `git clone https://github.com/ultralytics/yolov5` to update.
YOLOv5 🚀 v6.1-373-g8f28ea4 Python-3.7.13 torch-1.12.0+cu113 CUDA:0 (Tesla T4, 15110MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=

In [38]:
!python3 ./yolov5/detect.py --weights ./yolov5/runs/train/baseline_yolov5s_results3/weights/best.pt --img 416 --conf 0.25 --source ./data_for_yolo/data/images/test --save-txt --save-conf

detect: weights=['./yolov5/runs/train/baseline_yolov5s_results3/weights/best.pt'], source=./data_for_yolo/data/images/test, data=yolov5/data/coco128.yaml, imgsz=[416, 416], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_conf=True, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=yolov5/runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5 🚀 v6.1-373-g8f28ea4 Python-3.7.13 torch-1.12.0+cu113 CUDA:0 (Tesla T4, 15110MiB)

Fusing layers... 
Model summary: 213 layers, 7023610 parameters, 0 gradients, 15.8 GFLOPs
image 1/401 /content/drive/MyDrive/H_Tatarstan2022/data_for_yolo/data/images/test/0_23-11-2021_02-20-04_PM.jpg: 256x416 1 carplate, Done. (0.012s)
image 2/401 /content/drive/MyDrive/H_Tatarstan2022/data_for_yolo/data/images/test/0_23-11-2021_02-20-07_PM.jpg: 352x416 1 car, 1 carplate, Done. (0.012s)
ima

In [53]:
def get_soliton_labels_df(path_to_txt_folder):
    simple_solution = []
    for detection_file in os.listdir(path_to_txt_folder):
        img_name = detection_file.split('.')[0] + '.jpg'
        with open(path_to_txt_folder + detection_file, 'r') as f:
            data = f.read()
            data = [i for i in data.split('\n') if i != '']
        for line in data:
            val = [float(i) for i in line.split()]
            cls, xywh, conf = val[0], val[1:5], val[5]
            center_x, center_y, width, height = xywh
            xmin = center_x - (width / 2)
            xmax = center_x + (width / 2)
            ymin = center_y - (height / 2)
            ymax = center_y + (height / 2)
            simple_solution.append([img_name, int(cls), conf, xmin, xmax, ymin, ymax])
    return simple_solution

In [54]:
def get_test_labels_df(path_to_txt_folder):
    simple_solution = []
    for detection_file in os.listdir(path_to_txt_folder):
        img_name = detection_file.split('.')[0] + '.jpg'
        with open(path_to_txt_folder + detection_file, 'r') as f:
            data = f.read()
            data = [i for i in data.split('\n') if i != '']
        for line in data:
            val = [float(i) for i in line.split()]
            cls, center_x, center_y, width, height = val
            xmin = center_x - (width / 2)
            xmax = center_x + (width / 2)
            ymin = center_y - (height / 2)
            ymax = center_y + (height / 2)
            simple_solution.append([img_name, int(cls), xmin, xmax, ymin, ymax])
    return simple_solution

In [43]:
simple_solution = get_soliton_labels_df('./yolov5/runs/detect/exp2/labels/')
simple_solution = pd.DataFrame(simple_solution, columns=['ImageID', 'LabelName', 'Conf', 'XMin', 'XMax', 'YMin', 'YMax'])
simple_solution.head()

,ImageID,LabelName,Conf,XMin,XMax,YMin,YMax
0,0_23-11-2021_02-20-04_PM.jpg,4.0,0.910621,0.078948,0.172932,0.451724,0.620690
1,0_23-11-2021_02-20-07_PM.jpg,3.0,0.464965,0.663265,0.855102,0.519705,0.719211
2,0_23-11-2021_02-20-07_PM.jpg,4.0,0.797442,0.661225,0.857143,0.519705,0.716749
3,0_24-11-2021_01-13-52_PM.jpg,4.0,0.892228,0.842857,0.938775,0.449641,0.607913
4,0_24-11-2021_01-38-00_AM.jpg,4.0,0.925527,0.658537,0.835365,0.514019,0.724299


In [44]:
test_labels = get_test_labels_df('./data_for_yolo/data/labels/test/')
test_labels = pd.DataFrame(test_labels, columns=['ImageID', 'LabelName', 'XMin', 'XMax', 'YMin', 'YMax'])
test_labels.head()

,ImageID,LabelName,XMin,XMax,YMin,YMax
0,24-11-2021_02-42-31_PM.jpg,0.0,0.661849,0.682853,0.215546,0.271230
1,24-11-2021_02-42-31_PM.jpg,0.0,0.579472,0.595234,0.192487,0.269539
2,24-11-2021_02-42-31_PM.jpg,0.0,0.743170,0.752552,0.061691,0.101908
3,24-11-2021_02-42-31_PM.jpg,0.0,0.918542,0.923292,0.136447,0.167697
4,24-11-2021_02-42-31_PM.jpg,1.0,0.585618,0.592779,0.192671,0.206250


In [36]:
!pip install map-boxes

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [45]:
from map_boxes import mean_average_precision_for_boxes

mean_ap, average_precisions = mean_average_precision_for_boxes(test_labels, simple_solution, iou_threshold=0.5, verbose=False)
mean_ap

0.47968860431061244

## Предсказание для сайта

In [46]:
!python3 ./yolov5/detect.py --weights ./yolov5/runs/train/baseline_yolov5s_results3/weights/best.pt --img 416 --conf 0.25 --source ./data/test/images --save-txt --save-conf

detect: weights=['./yolov5/runs/train/baseline_yolov5s_results3/weights/best.pt'], source=./data/test/images, data=yolov5/data/coco128.yaml, imgsz=[416, 416], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_conf=True, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=yolov5/runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5 🚀 v6.1-373-g8f28ea4 Python-3.7.13 torch-1.12.0+cu113 CUDA:0 (Tesla T4, 15110MiB)

Fusing layers... 
Model summary: 213 layers, 7023610 parameters, 0 gradients, 15.8 GFLOPs
image 1/855 /content/drive/MyDrive/H_Tatarstan2022/data/test/images/0_23-11-2021_02-18-50_PM.jpg: 288x416 1 carplate, Done. (0.013s)
image 2/855 /content/drive/MyDrive/H_Tatarstan2022/data/test/images/0_23-11-2021_02-18-56_PM.jpg: 352x416 1 carplate, Done. (0.013s)
image 3/855 /content/drive/MyDrive/H_Tatarstan2022/d

In [55]:
simple_solution = get_soliton_labels_df('./yolov5/runs/detect/exp3/labels/')
simple_solution_df = pd.DataFrame(simple_solution, columns=['ImageID', 'LabelName', 'Conf', 'XMin', 'XMax', 'YMin', 'YMax'])
simple_solution_df.head()

,ImageID,LabelName,Conf,XMin,XMax,YMin,YMax
0,0_23-11-2021_02-18-50_PM.jpg,4,0.900941,0.091160,0.256906,0.665254,0.800848
1,0_23-11-2021_02-18-56_PM.jpg,4,0.913646,0.529167,0.791667,0.740000,0.850000
2,0_23-11-2021_02-19-19_PM.jpg,4,0.879377,0.833334,0.927928,0.620301,0.721805
3,0_23-11-2021_02-20-47_PM.jpg,4,0.925157,0.176471,0.367647,0.700565,0.824859
4,0_23-11-2021_02-37-47_PM.jpg,4,0.900057,0.544775,0.756219,0.735577,0.954327


In [56]:
simple_solution_df.describe()

,LabelName,Conf,XMin,XMax,YMin,YMax
count,10892.000000,10892.000000,1.089200e+04,10892.000000,1.089200e+04,10892.000000
mean,2.443904,0.635560,4.376536e-01,0.524194,2.857058e-01,0.392438
std,1.128374,0.228175,2.672965e-01,0.269538,1.815553e-01,0.218373
min,0.000000,0.250194,-2.000000e-07,0.008333,-2.000000e-07,0.023148
25%,3.000000,0.422962,2.124260e-01,0.295759,1.493422e-01,0.223027
50%,3.000000,0.655268,4.062500e-01,0.510938,2.539470e-01,0.354167
75%,3.000000,0.856112,6.578123e-01,0.744048,3.944445e-01,0.514974
max,4.000000,0.970077,9.911458e-01,1.000001,9.518520e-01,1.000001


In [50]:
# !mkdir solutions

In [57]:
simple_solution_df.to_csv("solutions/first_solution_baseline.csv",
                          sep=';', index=False)

### Проверка предсказания

In [58]:
path = 'solutions/first_solution_baseline.csv'

df = pd.read_csv(path, sep=';')
df.head()

,ImageID,LabelName,Conf,XMin,XMax,YMin,YMax
0,0_23-11-2021_02-18-50_PM.jpg,4,0.900941,0.091160,0.256906,0.665254,0.800848
1,0_23-11-2021_02-18-56_PM.jpg,4,0.913646,0.529167,0.791667,0.740000,0.850000
2,0_23-11-2021_02-19-19_PM.jpg,4,0.879377,0.833334,0.927928,0.620301,0.721805
3,0_23-11-2021_02-20-47_PM.jpg,4,0.925157,0.176471,0.367647,0.700565,0.824859
4,0_23-11-2021_02-37-47_PM.jpg,4,0.900057,0.544775,0.756219,0.735577,0.954327
